In [6]:
import requests
import json
import time
from kafka import KafkaProducer
from datetime import datetime 
import os
from dotenv import load_dotenv

ModuleNotFoundError: No module named 'kafka'

In [3]:
load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [5]:
print("Connecting to Kafka....")

producer = KafkaProducer(
    bootstrap_servers=['kafka:29092'],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

print("Connected to Kafka")

Connecting to Kafka....
Connected to Kafka


In [3]:
def get_authorization_token(client_id:str, client_secret: str):
    
    url = "https://auth.opensky-network.org/auth/realms/opensky-network/protocol/openid-connect/token"
    
    data = {
        "grant_type": "client_credentials",
        "client_id": client_id,
        "client_secret": client_secret
    }
    
    response = requests.post(url, data=data)

    if response.status_code == 200:

        return response.json()["access_token"]
    else:
        
        raise Exception(f"Ошибка: {response.status_code}, {response.text}")
    

In [1]:
def get_opensky_data(token):
    try:
        headers = {"Authorization": f"Bearer {token}"}
        
        url = "https://api.opensky-network.org/api/states/all"
        params = {
            'lamin': 35.0,   # минимальная широта
            'lamax': 70.0,   # максимальная широта
            'lomin': -10.0,  # минимальная долгота
            'lomax': 40.0    # максимальная долгота
        }
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()

        data = response.json()
        states = data.get('states',[])

        return states

    except Exception as e:
        print(e)

        return []

    

In [5]:
get_opensky_data(get_authorization_token(CLIENT_ID, CLIENT_SECRET))

NameError: name 'CLIENT_ID' is not defined